In [1]:
#============================= Economic evaluation of machine learning methods, graphical representation of excess return of machine learning portfolio, grahical represention of DOC-GARCH(1,1) features=====================================#
#=============================================================================================================================================================================================================================================#


!pip install dask
!pip install numpy
!pip install pandas
!pip install "plotly>=4" numpy pandas
!pip install -U kaleido
!pip install pyfolio

import plotly.graph_objects as go
import dask.dataframe as dd
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from timeit import default_timer as timer
import warnings
from pandas import plotting
from typing import Tuple
import pyfolio as pf
import math
%matplotlib inline
import os

if not os.path.exists("images"):
    os.mkdir("images")
warnings.filterwarnings("ignore")



In [2]:
# Import ouf-of-sample R^2 results based on machine learning methods

file=pd.read_csv(r'C:/Users/Emile Ndoumbe/Dropbox/Oo_R_sq.csv')

In [3]:
path1 = r'C:/Users/Emile Ndoumbe/Dropbox/dossier_machinelearning_data/sDOC_results'                     
all_files1 = glob.glob(path1 + "/*.csv")

path2 = r'C:/Users/Emile Ndoumbe/Dropbox/dossier_machinelearning_data/PCRresult'                     
all_files2 = glob.glob(path2 + "/*.csv")


path3 = r'C:/Users/Emile Ndoumbe/Dropbox/dossier_machinelearning_data/Rdomfo_results'                     
all_files3 = glob.glob(path3 + "/*.csv")

path5 = r'C:/Users/Emile Ndoumbe/Dropbox/dossier_machinelearning_data/NNfresult'                     
all_files5 = glob.glob(path5 + "/*.csv")

# Gain in terms of sharpe ratio

In [5]:
### Gain in terms of sharpe ratio for the sDOC method

# Set the current working directory

path1 = r'C:/Users/Emile Ndoumbe/Dropbox/dossier_machinelearning_data/sDOC_results'                     
all_files1 = glob.glob(path1 + "/*.csv")

# Gain in terms of sharpe ratio
sharpe_ratio_gain1=[]
for i in range(63):
    fil=pd.read_csv(all_files1[i])
    sharpe=[]
    sharp=[]
    for j in range(fil.shape[0]):
        SR_0=(fil.iloc[j,6])/math.sqrt(fil.iloc[j,4])
        SR=(fil.iloc[j,1])/math.sqrt(fil.iloc[j,2])
        sharpe.append(SR_0)
        sharp.append(SR)
    sharpe_1=np.mean(sharpe)
    shp=np.mean(sharp)
    
    #sharpe_2=(sharpe_1+file.iloc[i,3])/(1-file.iloc[i,3])
    #SR_1=math.sqrt(sharpe_2)- shp
    SR_1=shp-sharpe_1
    
    sharpe_ratio_gain1.append(SR_1)
    



In [263]:
### Gain in terms of sharpe ratio for the PCA method

# Set the current working directory

path2 = r'C:/Users/Emile Ndoumbe/Dropbox/dossier_machinelearning_data/PCRresult'                     
all_files2 = glob.glob(path2 + "/*.csv")

# Gain in terms of sharpe ratio

sharpe_ratio_gain2=[]
for i in range(63):
    fil=pd.read_csv(all_files2[i])
    sharpe=[]
    sharp=[]
    for j in range(fil.shape[0]):
        SR_0=(fil.iloc[j,6]**2)/fil.iloc[j,4]
        SR=(fil.iloc[j,6])/math.sqrt(fil.iloc[j,4])
        sharpe.append(SR_0)
        sharp.append(SR)
    sharpe_1=np.mean(sharpe)
    shp=np.mean(sharp)
    
    sharpe_2=(sharpe_1+file.iloc[i,3])/(1-file.iloc[i,3])
    SR_1=math.sqrt(sharpe_2)- shp
    sharpe_ratio_gain2.append(SR_1)    

In [255]:
### Gain in terms of sharpe ratio for the random forest method

# Set the current working directory

path3 = r'C:/Users/Emile Ndoumbe/Dropbox/dossier_machinelearning_data/Rdomfo_results'                     
all_files3 = glob.glob(path3 + "/*.csv")

sharpe_ratio_gain3=[]
for i in range(63):
    fil=pd.read_csv(all_files3[i])
    sharpe=[]
    sharp=[]
    for j in range(fil.shape[0]):
        SR_0=(fil.iloc[j,6]**2)/fil.iloc[j,4]
        SR=(fil.iloc[j,6])/math.sqrt(fil.iloc[j,4])
        sharpe.append(SR_0)
        sharp.append(SR)
    sharpe_1=np.mean(sharpe)
    shp=np.mean(sharp)
    
    sharpe_2=(sharpe_1+file.iloc[i,3])/(1-file.iloc[i,3])
    SR_1=math.sqrt(sharpe_2)- shp
    sharpe_ratio_gain3.append(SR_1)

In [23]:
### Gain in terms of sharpe ratio for the neural network method

# Set the current working directory

path5 = r'C:/Users/Emile Ndoumbe/Dropbox/dossier_machinelearning_data/NNfresult'                     
all_files5 = glob.glob(path5 + "/*.csv")

# Gain in terms of sharpe ratio
sharpe_ratio_gain5=[]
for i in range(63):
    fil=pd.read_csv(all_files5[i])
    sharpe=[]
    for j in range(fil.shape[0]):
        SR_0=(fil.iloc[j,6]**2)/fil.iloc[j,4]
        sharpe.append(SR_0)
    sharpe_1=np.mean(sharpe)
    sharpe_2=(sharpe_1+file.iloc[i,4])/(1-file.iloc[i,4])
    SR_1=math.sqrt(sharpe_2)- math.sqrt(sharpe_1) 
    
    sharpe_ratio_gain5.append(SR_1)

# Gain in terms of average excess portfolio return, average utility, transaction cost, turnover and maximum drawdown

In [2359]:
## Function that performs a division: if the denominator is zero, the quotient of the division is fixed to zero
def safe_div(x,y):
    if y == 0:
        return 0
    return x / y

## Excess portfolio return series for machine learning method, excess portfolio return for a buy-and-hold investor,
## Utility values for machine learning method, utilility series for a buy-and-hold investor, monthly turnover series,

vector_1=[]
vector_2=[]
vector_3=[]
vector_4=[]
vector_5=[]
vector_6=[]

fil=pd.read_csv(all_files1[0])
fil=fil.fillna(0)
a=fil.shape[0]

# Investor with a small relative risk aversion
gamma=3

for i in range(a):
    x=fil.iloc[i,1] # vector of out-of-sample forecasts 
    y=fil.iloc[i,2] # vector of one-period ahead predicted volatility
    f=fil.iloc[i,5] # vector of rolling average of excess industry-level return
    g=fil.iloc[i,4] # vector of rolling variance of excess industry-level return
    l=fil.iloc[i,3] # vector of observed excess industry-level returns 
    e=safe_div(x,y) # vector of out-of-sample forecasts divided by one-period ahead predicted volatility
    
    # Optimal portfolio weight
    w=e/gamma 
    
    # Utility corresponding to a machine learning method (after simplification)
    u_1=(w*x)/2 
    
    # Excess portfolio return using a machine learning method
    
    u_2=w*l 
    
    # vector of rolling average of excess industry-level return divided by the rolling variance of excess industry-level return
        
    h=safe_div(f,g)
    
    # Vector of weight corresponding to a buy-and-hold strategy
    v=h/gamma
    
    # Utility corresponding to a buy-and-hold investor (after simplification)
    m_1=(v*f)/2   
    
    # Excess portfolio return for a buy-and-hold investor
    
    m_2=v*l 
    vector_1.append(w)
    vector_2.append(v)
    vector_3.append(u_1)
    vector_4.append(u_2)
    vector_5.append(m_1)
    vector_6.append(m_2)

In [2360]:
# Average utility for a machine learning method
dd=sum(vector_3)/len(vector_3) 

# Average utility for a buy-and-hold investor
bb=sum(vector_5)/len(vector_5)

# Average portfolio return for a machine learning method
cc=sum(vector_4)/len(vector_4)

#Average portfolio return for a buy-and-hold investor
ee=sum(vector_6)/len(vector_6)

# Series of excess portfolio return
D=pd.Series(vector_4)

# Vector of montly turnover
I=pd.DataFrame(vector_1)
e=abs(I-I.shift(1)) 

# Average momthly turnover
X= np.mean(e) 

# Transaction cost

X_1= dd-bb 

# Gain in term of average portfolio return
X_2= cc-ee

DD=pd.DataFrame(vector_4)
DD.columns=['raw_ret']


In [ ]:
## Maximum drawdom

# Series of excess portfolio return associated with date

date=pd.period_range('2013-3-29', periods=93, freq='M')
data = pd.Series(D.values, index=date)

# Maximum drawdown values

pf.create_returns_tear_sheet(data)

# Gain to pain ratio, Calmar ratio and Sortino ratio

In [2333]:

def annualized_return(df: pd.DataFrame, ret_column = 'raw_ret'):
    '''Compute Annualized Returns
    INPUT 
        df: a dataframe containing a column of raw returns
    OUTPUT
        a float number of annualized return
    Reference: https://www.investopedia.com/terms/a/annualized-total-return.asp#:~:text=An%20annualized%20total%20return%20is,the%20annual%20return%20was%20compounded.
                        and https://www.assetmacro.com/financial-terms/annualized-return/#:~:text=Annualized%20Return%20is%20the%20average,252%20trading%20days%20of%20year).
    '''
    gross_return = (1 + df[ret_column]).prod()
    months = df.shape[0]
    years = months / 12 # there are about 12 months per year 
    ann_return = gross_return **(1/years)
    ann_return = ann_return - 1
    return ann_return



def gain_to_pain_ratio(df: pd.DataFrame, ret_column = 'raw_ret'):
    ''' Calculate Schwager's Gain to Pain Ratio
    INPUT 
        df: a dataframe containing a column of raw returns
    OUTPUT
        a float number of the gain-to-pain ratio (a Gain to Pain ratio above 1.25 is considered good, and a value over 2 is very good.)
    Reference: https://www.investmacro.com/forex/2018/07/5-statistics-for-analyzing-your-trading-performance 
                        and https://jackschwager.com/market-wizards-search-part-2-the-performance-statistics-i-use '''

    net_return = df[ret_column].sum()
    abs_negative_return = abs( df[ret_column][df[ret_column] < 0].sum() )
    gain_to_pain = net_return / (abs_negative_return + 1e-4)
    return gain_to_pain

def calmar_ratio(df: pd.DataFrame, ret_column = 'raw_ret'):
    '''Annualized Return over Max Drawdown
    INPUT 
        df: a dataframe containing a column of raw returns
    OUTPUT
        a float number of the Calmar ratio (a Calmar ratio of 0.50 is minimal. Anything over 1.0 is considered a pretty healthy risk adjusted return.)
    Reference: Terry W Young. Calmar ratio: A smoother tool. Futures, 20(1):40, 1991. '''

    calmar = annualized_return(df) / (max_drawdown + 1e-4)
    return calmar

def sortino_ratio(df: pd.DataFrame, ret_column = 'raw_ret'):
    '''Annualized Return - RF rate / Annualized Downside Standard Deviation
    INPUT 
        df: a dataframe containing a column of raw returns and a column of risk-free rates
    OUTPUT
        a float number of the Sortino ratio
    Reference: https://www.investmacro.com/forex/2018/07/5-statistics-for-analyzing-your-trading-performance  '''

    # Calculate the annualized average daily excess return
    excess_returns = df[ret_column]
    ann_mean = excess_returns.mean() * 12  # there are about 12 months per year

    # Calculate the annualized daily standard deviation of negative excess returns
    downside_excess_returns = excess_returns[excess_returns < 0]
    ann_downside_std_dev = downside_excess_returns.std() * (12 ** 0.5)
   
    return ann_mean / ann_downside_std_dev


# Cumulative excess return of machine learning portfolio

In [ ]:
# Cumulative excess portfolio return : for each method, compute a cumulative excess return cum_sdoc, cum_pcr, cum_RF, cum_NN3

cum_sdoc=D.cumsum() # We can compute the rest of cumulative excess return by changing the method


a_0=pd.DataFrame(fil.iloc[:,0])

# Put the cumulative excess returns of machine learning portfolio in the same file

file=pd.concat([a_0,cum_sdoc,cum_pca,cum_RF,cum_NN3],axis=1)
file.columns=['date','sdoc','pcr','RF','NN3']

# Graphical representation of cumulative excess return of machine learning portfolio
fig=go.Figure()
trace6=go.Scatter(x=file['date'],y=file['sdoc'],name='sDOC')
trace7=go.Scatter(x=file['date'],y=file['pcr'],name='PCR')
trace8=go.Scatter(x=file['date'],y=file['RF'],name='RF')
trace9=go.Scatter(x=file['date'],y=file['NN3'],name='NN3')

fig.add_trace(trace6)
fig.add_trace(trace7)
fig.add_trace(trace8)
fig.add_trace(trace9)

fig.show()
fig.write_image('images/ind13.pdf')

# Graphical representation of DOC-GARCH features

In [ ]:
# Set the path

path = r'C:/Users/Emile Ndoumbe/Dropbox/garch_month'                     
all_files = glob.glob(path + "/*.csv")


fig=go.Figure()
trace6=go.Scatter(x=file['date'],y=file['X1'],name='garch_doc1')
trace7=go.Scatter(x=file['date'],y=file['X2'],name='garch_doc2')
trace8=go.Scatter(x=file['date'],y=file['X3'],name='garch_doc3')
trace9=go.Scatter(x=file['date'],y=file['X4'],name='garch_doc4')
trace10=go.Scatter(x=file['date'],y=file['X5'],name='garch_doc5')
fig.add_trace(trace6)
fig.add_trace(trace7)
fig.add_trace(trace8)
fig.add_trace(trace9)
fig.add_trace(trace10)
fig.show()
fig.write_image('images/garchdoc_63_1.pdf')